In [2]:
import json, os
def open_json(path):
    with open(path, "r", encoding='ISO-8859-1') as f:
        data = json.load(f)
    return data

def save_json(a, fn):
    dir_path = os.path.dirname(fn)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    b = json.dumps(a)
    f2 = open(fn, 'w')
    f2.write(b)
    f2.close()

In [27]:
class GV:

    NAMES = {
        'EXT_COL': 'extract_column', #!
        'CAL_COL': 'calculate_column', #!
        'BOOL_COL': 'boolean_column', #!
        'COMB_COL': 'combine_column', #!
        'INF_COL': 'infer_column', #!
        'END': 'end', #!
        'GEN_NEW_COL': 'generate_new_column', #!


        'STAND': 'standardization',
        
        'EXT_ROW': 'extract_row',
        'EXT_MAX_CONS_RECORD': 'extract_max_consecutive_record',

        'SORT_BY': 'sort_by',
        'GROUP_STATISTICS': 'group_statistics',

        'GEN_CON_COL': 'generate_conditional_column',
        'SPLIT_COL': 'split_column',

        'INIT': '<init>',
        'END': '<assign_other_agent>',

        'STAND_DATETIME': 'standardize_datetime',
        'REMOVE_SYMBOL': 'remove_noisy_symbol',
        'REMOVE_UNIT': 'remove_unit',
        'STAND_NUMERICAL': 'standardize_numerical',
        'FILTER_COLUMNS': 'filter_columns',
    }

In [41]:
#!-----------------------------------Construct Prompt for TabQA-------------------------------------------- 
prompt = {
    'description': """This agent is designed to deduce the missing value of the column(s) in the table. 

If some values can not be deduced, please keep the original value. If some values are ambiguous, please deduce a reasonable value to try not to affect the execution of the SQL query.""",

    'normalized_demos': [

        """/*
{
    1: {"rank": "nan", "rank_cleaned": "[?]"},
    2: {"rank": "nan", "rank_cleaned": "[?]"},
    3: {"rank": "nan", "rank_cleaned": "[?]"},
    4: {"rank": "4.0", "rank_cleaned": 4},
    5: {"rank": "5.0", "rank_cleaned": 5},
    6: {"rank": "6.0", "rank_cleaned": 6},
}
*/
Requirement: please standardize the column `rank` to numerical format abd fill the token [?] of row(1, 2, 3).
SQL: SELECT rank FROM table WHERE rank < 10
Output: ```{1: {'rank': 'nan', 'rank_cleaned': '[n.a.]'}, 2: {'rank': 'nan', 'rank_cleaned': 2}, 3: {'rank': 'nan', 'rank_cleaned': 3}}```""",

        """/*
{
	1: {'year_built': 1966, 'year_built_cleaned': 1966},
	2: {'year_built': 1984, 'year_built_cleaned': 1984},
	3: {'year_built': 1931, 'year_built_cleaned': 1931},
	4: {'year_built': 'Early 2012', 'year_built_cleaned': '[?]'},
	5: {'year_built': '194?', 'year_built_cleaned': '[?]'},
}
*/
Requirement: please standardize the column `year_built` to numerical format and fill the token [?] of row(4, 5).
SQL: SELECT year_built FROM table WHERE year_built > 1935
Output: ```{4: {'year_built': 'Early 2012', 'year_built_cleaned': 2012}, 5: {'year_built': '194?', 'year_built_cleaned': 1940}}```""",

        """/*
{
	1: {'position': '4th', 'position_cleaned': 4},
	2: {'position': '4th', 'position_cleaned': 4},
	3: {'position': '3rd', 'position_cleaned': 3},
	4: {'position': 'nan', 'position_cleaned': '[?]'},
	5: {'position': '8th', 'position_cleaned': 8},
	6: {'position': 'Champion', 'position_cleaned': '[?]'},
	7: {'position': '6th', 'position_cleaned': 6},
	8: {'position': 'Champion', 'position_cleaned': '[?]'},
}
*/
Requirement: please standardize the column `position` to numerical format and fill the token [?] of row(4, 6, 8).
SQL: SELECT `position` FROM table WHERE `position` < 3
Output: ```{4: {'position': 'nan', 'position_cleaned': '[n.a.]'}, 6: {'position': 'Champion', 'position_cleaned': 1}, 8: {'position': 'Champion', 'position_cleaned': 1}}```""",

        """/*
{
	1: {'time': '10.55', 'time_cleaned': 10.55},
	2: {'time': '11.00', 'time_cleaned': 11.0},
	3: {'time': '11.17', 'time_cleaned': 11.17},
	4: {'time': 'dns', 'time_cleaned': '[?]'},
	5: {'time': 'dp', 'time_cleaned': '[?]'},
}
*/
Requirement: please standardize the column `time` to numerical format and fill the token [?] of row(4, 5).
SQL: SELECT `name` FROM w WHERE `time` = (SELECT MIN(`time`) FROM w WHERE `heat` = 1)
Output: ```{4: {'time': 'dns', 'time_cleaned': '[n.a.]'}, 5: {'time': 'dp', 'time_cleaned': '[n.a.]'}}```""",

        """/*
{
	1: {'diameter': '18 mm', 'diameter_cleaned': 18},
	2: {'diameter': '21 mm', 'diameter_cleaned': 21},
	3: {'diameter': '24 mm', 'diameter_cleaned': 24},
    4: {'diameter': '32-33 mm', 'diameter_cleaned': '[?]'},
}
Requirement: please standardize the column `diameter` to numerical format and fill the token [?] of row(4).
SQL: SELECT `diameter` FROM w WHERE `diameter` > 20
Output: ```{4: {'diameter': '32-33 mm', 'diameter_cleaned': 32}}```""",
    ],

    'augment_demos': [
        

        """/*
{
	1: {'score': 'w 8-1', 'score_difference': 7},
	2: {'score': 'w 4-3 (10)', 'score_difference': 1},
	3: {'score': 'w 6:5', 'score_difference': '[?]'},
	4: {'score': 'l 2-5', 'score_difference': '3'},
	5: {'score': 'l4-8', 'score_difference': '[?]'},
}
*/
Requirement: please generate a new column `score_difference` based on the column `score` and fill the token [?] of row(3, 5).
SQL: SELECT COUNT(*) FROM w WHERE `score_difference` = 1
Output: ```{3: {'score': 'w 6:5', 'score_difference': 1}, 5: {'score': 'l4-8', 'score_difference': 4}}```""",

        """/*
{
	1: {'launched': '19 february 1946', 'fate': 'scrapped in 1972', 'service_duration': '26'},
    2: {'launched': '30 august 1946', 'fate': 'scrapped in 1974', 'service_duration': '26'},
    3: {'launched': '6 july 1944', 'fate': 'sold as oil hulk in 1960', 'service_duration': '[?]'},
    4: {'launched': '1943/10/27', 'fate': 'scrapped in 1960', 'service_duration': '[?]'},
    5: {'launched': '30 september 1943', 'fate': 'scrapped in 1966', 'service_duration': '17'},
}
*/
Requirement: please generate a new column `service_duration` based on the columns `launched` and `fate` and fill the token [?] of row(3, 4).
SQL: SELECT MAX(`service_duration`) FROM w WHERE `service_duration` < 20
Output: ```{3: {'launched': '6 july 1944', 'fate': 'sold as oil hulk in 1960', 'service_duration': 16}, 4: {'launched': '1943/10/27', 'fate': 'scrapped in 1960', 'service_duration': 17}}```""",

        """/*
{
	1: {'season': '2000-01', 'season_cleaned': '2000-2001'},
	2: {'season': '2005-06', 'season_cleaned': '2005-2006'},
	3: {'season': '2009-10', 'season_cleaned': '[?]'},
	4: {'season': '2010-11', 'season_cleaned': '[?]'},
	5: {'season': '2011-12', 'season_cleaned': '[?]'},
}
*/
Requirement: please generate a new column `season_cleaned` based on the column `season` and fill the token [?] of row(3, 4, 5).
SQL: SELECT MIN(`season`) FROM w WHERE `league_top_scorer` LIKE '%bent christensen%'
Output: ```{3: {'season': '2009-10', 'season_cleaned': '2009-2010'}, 4: {'season': '2010-11', 'season_cleaned': '2010-2011'}, 5: {'season': '2011-12', 'season_cleaned': '2011-2012'}}```""",

    ],

    'normalize_query': """Please complete the following prompt.

/*
{table}
*/
Requirement: please standardize the column `{column}` to {coltype} format and fill the token [?] of row({rows}).
SQL: {sql}
Please output json format as above. If the value cannot be deduced, keep the original value.
Output:""",

    'augment_query': """Please complete the following prompt.
    
/*
{table}
*/
Requirement: please generate a new column `{new_col}` based on the {column_flag} `{column_str}` and fill the token [?] of row({rows}).
SQL: {sql}
Please output json format as above. If the value cannot be deduced, output [n.a.].
Output:""",

    'self_correction': """/*
{context}
*/
Requirement: {question}
Last Error: {last_error}
Output: {a}"""
}

save_json(prompt, "./agent-imputater.json")

#!-------------------------------------------------------------------------------



In [13]:
d = open_json("./tablefact/agent-manager.json")

In [15]:
print(d['demos'][0])

Title: 2007-08 NHL season
/*
date	team_a	team_b	place
2007-10-4	1	2	home
2008-1-1	1	3	home
2013-5-1	1	4	home
*/
Q: which game have the largest score difference?
Requirements: ```{(1) related columns: team_a, team_b; (2) generate a column about the score difference from the columns team_a, team_b; (3) please normalize the columns team_a to int format; please normalize the columns team_b to int format;}```
